In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

In [2]:
using System.IO;

var connStr = File.ReadAllText(@"C:\Users\Pedro_Carvalho1\conn_str.txt");

#!share --from csharp connStr

In [ ]:
#!connect mssql --kernel-name sql --connection-string @connStr

In [ ]:
#!sql-sql
-- Garantir que o schema existe
if not exists (select * from sys.schemas where name = 'groupex')
    exec('create schema groupex');
go

-- Se a tabela já existir, elimina antes
if object_id('groupex.Sales', 'U') is not null
    drop table groupex.Sales;
go

-- Criação da tabela
create table groupex.Sales (
    sale_id int primary key,
    product_id int not null,
    category varchar(50) not null,
    sale_date date not null,
    quantity int not null,
    unit_price decimal(10,2) not null,
    discount_applied bit not null -- 0 = sem desconto, 1 = com desconto
);


In [ ]:
#!sql-sql

insert into groupex.Sales (sale_id, product_id, category, sale_date, quantity, unit_price, discount_applied)
values
(1, 101, 'Electronics', '2023-01-05', 2, 500.00, 0),
(2, 102, 'Electronics', '2023-01-05', 1, 1200.00, 1),
(3, 103, 'Clothing',    '2023-01-06', 5, 50.00, 0),
(4, 104, 'Clothing',    '2023-01-07', 2, 80.00, 1),
(5, 105, 'Clothing',    '2023-01-07', 1, 120.00, 1),
(6, 106, 'Electronics', '2023-01-08', 3, 300.00, 0),
(7, 107, 'Books',       '2023-01-08', 4, 25.00, 1),
(8, 108, 'Books',       '2023-01-09', 2, 40.00, 0);


In [9]:
#!sql-sql
select * from groupex.Sales

(8 rows affected)

sale_id,product_id,category,sale_date,quantity,unit_price,discount_applied
1,101,Electronics,2023-01-05 00:00:00Z,2,500.00,False
2,102,Electronics,2023-01-05 00:00:00Z,1,1200.00,True
3,103,Clothing,2023-01-06 00:00:00Z,5,50.00,False
4,104,Clothing,2023-01-07 00:00:00Z,2,80.00,True
5,105,Clothing,2023-01-07 00:00:00Z,1,120.00,True
6,106,Electronics,2023-01-08 00:00:00Z,3,300.00,False
7,107,Books,2023-01-08 00:00:00Z,4,25.00,True
8,108,Books,2023-01-09 00:00:00Z,2,40.00,False


### ❓ Pergunta

Monte uma query que traga, **por categoria e por dia**:

1. Quantidade total de vendas (`sales_count` = número de registros).
2. Quantidade total de itens vendidos (`total_quantity`).
3. Valor total vendido (`total_amount` = quantidade × preço unitário).
4. Quantidade de vendas com desconto (`discounted_sales_count`).
5. Valor total vendido com desconto (`discounted_total_amount`).

👉 Use **agrupamento (`GROUP BY`) + `SUM(CASE WHEN …)`** para calcular as métricas de desconto.
👉 Não use CTEs, nem Window Functions, nem subqueries – só agregações.

---

⚡ Assim você vai treinar exatamente o estilo que comentou: **`SUM/COUNT` com `CASE` no mesmo agrupamento**.

Quer que eu já prepare uma **segunda questão mais puxada** (com `JOIN` entre duas tabelas e filtros condicionais), ou prefere primeiro resolver essa e me mostrar seu `SELECT`?


In [ ]:
#!sql-sql

SELECT
    format( sale_date, 'yyyy-MM-dd') as DAY
    ,category AS CATEGORY
    ,COUNT(*) AS SALES_COUNT
    ,SUM( QUANTITY ) AS TOTAL_QUANTITY
    ,SUM( QUANTITY * UNIT_PRICE ) AS TOTAL_AMOUNT
    ,SUM( CASE WHEN DISCOUNT_APPLIED = 1 THEN 1 ELSE 0 END ) AS DISCOUNTED_SALES_COUNT
    ,SUM( CASE WHEN DISCOUNT_APPLIED = 1 THEN QUANTITY * UNIT_PRICE ELSE 0 END ) AS DISCOUNTED_TOTAL_AMOUNT
FROM   
    groupex.sales
GROUP BY
    format(sale_date, 'yyyy-MM-dd')
    ,category


(6 rows affected)

DAY,CATEGORY,SALES_COUNT,TOTAL_QUANTITY,TOTAL_AMOUNT,DISCOUNTED_SALES_COUNT,DISCOUNTED_TOTAL_AMOUNT
2023-01-08,Books,1,4,100.00,1,100.00
2023-01-09,Books,1,2,80.00,0,0.00
2023-01-06,Clothing,1,5,250.00,0,0.00
2023-01-07,Clothing,2,3,280.00,2,280.00
2023-01-05,Electronics,2,3,2200.00,1,1200.00
2023-01-08,Electronics,1,3,900.00,0,0.00
